# (Homework) Week 6 - DataScience Bootcamp Fall 2025

All solution cells are replaced with `# TODO` placeholders so you can fill them in.

**Name:** \
**Email:**

---

### Problem 1: Dataset Splitting

1. You have recordings of 44 phones from 100 people; each person records ~200 phones/day for 5 days.
   - Design a valid training/validation/test split strategy that ensures the model generalizes to **new speakers**.

2. You now receive an additional dataset of 10,000 phone recordings from **Kilian**, a single speaker.
   - You must train a model that performs well **specifically for Kilian**, while also maintaining generalization.

*Describe your proposed split strategy and reasoning.* (Theory)

In [ ]:
"""Problem 1 – Dataset Splitting

(1) How I would split the original 100-speaker dataset

To evaluate generalization to *new* speakers, we must split the data at the **speaker level**, not at the recording level.

A simple speaker-level split:
- Train: 70 speakers
- Validation: 15 speakers
- Test: 15 speakers

For each speaker, all of their recordings (all 5 days and all phones) are placed into exactly one split. No speaker appears in more than one split. This way, test accuracy reflects how well the model generalizes to completely unseen speakers.

(2) How to handle Kilian’s 10,000 additional recordings

Now we have:
- The original 100-speaker dataset.
- A large single-speaker dataset from Kilian (10,000 recordings).

We care about:
- Performance on “average” new speakers.
- Performance specifically for Kilian.

A reasonable strategy:

1. Keep the speaker-independent split from part (1) for the 100 speakers.
   - Train, validation, and test speakers remain disjoint.

2. Create a separate split for Kilian’s data, for example:
   - Kilian-train: 70% of his recordings (or 3 of 5 days).
   - Kilian-val:   15% (or 1 day).
   - Kilian-test:  15% (or 1 day).

3. Training:
   - Train on the union of:
     • Original training speakers (all their recordings), and
     • Kilian-train recordings.
   - During development, monitor two validation sets:
     • Original validation speakers → general performance.
     • Kilian-val → performance on this specific user.

4. Final evaluation:
   - Report two test metrics:
     • Original test speakers → speaker-independent generalization.
     • Kilian-test → personalized performance for Kilian.

This setup both preserves a clean evaluation of generalization to new speakers and gives the model extra data to perform well on Kilian.
"""


### Problem 2: K-Nearest Neighbors

1. **1-NN Classification:** Given dataset:

   Positive: (1,2), (1,4), (5,4)

   Negative: (3,1), (3,2)

   Plot the 1-NN decision boundary and classify new points visually.

2. **Feature Scaling:** Consider dataset:

   Positive: (100,2), (100,4), (500,4)

   Negative: (300,1), (300,2)

   What would the 1-NN classify point (500,1) as **before and after scaling** to [0,1] per feature?

3. **Handling Missing Values:** How can you modify K-NN to handle missing features in a test point?

4. **High-dimensional Data:** Why can K-NN still work well for images even with thousands of pixels?


In [ ]:
"""Problem 2 – K-Nearest Neighbors

2.1 1-NN decision boundary

Positive points: (1,2), (1,4), (5,4)
Negative points: (3,1), (3,2)

The 1-NN classifier labels each location in the plane by the class of the single closest training point under Euclidean distance. The decision boundary is formed by the perpendicular bisectors between points with different labels, restricted to their Voronoi cells.

Below, I visualize the decision regions by sampling a grid of points, assigning each grid point to its nearest neighbor, and plotting the result.

2.2 Effect of feature scaling

We compare 1-NN predictions for the query (500, 1) before and after scaling features to [0, 1].

Before scaling:

Distances to positives:
- d((500,1),(100,2))  ≈ 400.00
- d((500,1),(100,4))  ≈ 400.01
- d((500,1),(500,4))  = 3

Distances to negatives:
- d((500,1),(300,1))  = 200
- d((500,1),(300,2))  ≈ 200.00

The nearest neighbor is (500,4), a positive point → prediction: POSITIVE.

After min–max scaling using the training data:

For feature x:
- min_x = 100, max_x = 500 → x' = (x − 100) / 400

For feature y:
- min_y = 1, max_y = 4   → y' = (y − 1) / 3

The scaled query becomes (1.0, 0.0). Its nearest neighbor becomes (300,1), which is negative → prediction: NEGATIVE.

This shows that feature scaling can completely change the nearest neighbor when one feature has a much larger range than the other.

2.3 Handling missing values in K-NN

If some features of a test point are missing, one simple modification is:

- When computing the distance between a test point x and a training point x_i, only use the coordinates where x is observed (and x_i has a value).
- Optionally normalize the distance by the number of used coordinates so distances remain comparable.

Another common option is to impute missing features (e.g., mean or median per feature) and then run standard K-NN.

2.4 Why K-NN can work for images (high-dimensional data)

Images have thousands of pixel features, so the ambient dimension is high. However, natural images occupy a much lower-dimensional manifold: they have structure, local correlations, and class-dependent patterns. Images from the same class tend to cluster together in pixel or embedding space.

With enough training examples, and especially when using good feature representations (e.g., embeddings from a neural network), the nearest neighbors of an image are often visually and semantically similar. Therefore, K-NN can still work well in these high-dimensional but structured spaces.
"""

# Visualization code for Problem 2.1
import numpy as np
import matplotlib.pyplot as plt

# training points
pos = np.array([[1, 2], [1, 4], [5, 4]])
neg = np.array([[3, 1], [3, 2]])

X = np.vstack([pos, neg])
y = np.array([1, 1, 1, -1, -1])

# grid over the plane
xx, yy = np.meshgrid(np.linspace(0, 6, 200), np.linspace(0, 5, 200))
grid = np.c_[xx.ravel(), yy.ravel()]

# 1-NN predictions
pred = []
for g in grid:
    dists = np.linalg.norm(X - g, axis=1)
    nn_idx = np.argmin(dists)
    pred.append(y[nn_idx])
pred = np.array(pred).reshape(xx.shape)

plt.figure()
plt.contourf(xx, yy, pred, alpha=0.2)
plt.scatter(pos[:, 0], pos[:, 1], marker='o', label='positive')
plt.scatter(neg[:, 0], neg[:, 1], marker='x', label='negative')
plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Problem 2.1 – 1-NN Decision Boundary")
plt.legend()
plt.show()


### Problem 3: Part 1

You are given a fully trained Perceptron model with weight vector **w**, along with training set **D_TR** and test set **D_TE**.

1. Your co-worker suggests evaluating $h(x) = sign(w \cdot x)$ for every $(x, y)$ in D_TR and D_TE. Does this help determine whether test error is higher than training error?
2. Why is there no need to compute training error explicitly for the Perceptron algorithm?

In [ ]:
"""Problem 3 – Part 1: Perceptron training vs test error

We are given a trained Perceptron with weight vector w and two datasets: a training set D_TR and a test set D_TE.

(1) Using h(x) = sign(w · x) to compare training and test error

We can apply h(x) = sign(w · x) to every example in both D_TR and D_TE:

- For each (x, y) in D_TR:
  • Predict y_hat = sign(w · x).
  • Count misclassifications where y_hat != y.
  • Training error = (number of misclassified training examples) / |D_TR|.

- For each (x, y) in D_TE:
  • Predict y_hat = sign(w · x).
  • Test error = (number of misclassified test examples) / |D_TE|.

Comparing these two error rates tells us whether the model is overfitting (e.g., very low training error but noticeably higher test error).

(2) Why we do not need to explicitly compute training error for the Perceptron

For linearly separable data, the Perceptron algorithm updates w only when it misclassifies a training point. It stops when it can make a full pass through the training set with no misclassifications.

At convergence:
- Every training example satisfies y (w · x) > 0.
- Therefore, the training error is exactly 0.

Because of this property, we do not need to run a separate pass to compute the training error: the fact that the algorithm has converged already guarantees zero training error on D_TR (assuming separability).
"""


### Problem 3: Two-point 2D Dataset (Part 2)

Run the Perceptron algorithm **by hand or in code** on the following data:

1. Positive class: (10, -2)
2. Negative class: (12, 2)

Start with $w_0 = (0, 0)$ and a learning rate of 1.

- Compute how many updates are required until convergence.
- Write down the sequence of $w_i$ vectors.

In [ ]:
"""Problem 3 – Two-point 2D Dataset (Part 2)

Data:
- Positive point x_pos = (10, -2) with label y = +1
- Negative point x_neg = (12,  2) with label y = -1

We run the Perceptron with learning rate 1 and initial weight
w0 = (0, 0),
and we iterate over the data in the fixed order: first x_pos, then x_neg.

Updates:

w0 = (0, 0)

1st pass:
- x_pos: y (w0 · x_pos) = 1 * (0) = 0 → misclassified
  w1 = w0 + y x_pos = (0, 0) + (10, -2) = (10, -2)

- x_neg: w1 · x_neg = 10*12 + (-2)*2 = 120 - 4 = 116, y = -1
  y (w1 · x_neg) = -1 * 116 = -116 ≤ 0 → misclassified
  w2 = w1 + y x_neg = (10, -2) + (-12, -2) = (-2, -4)

2nd pass:
- x_pos: w2 · x_pos = (-2)*10 + (-4)*(-2) = -20 + 8 = -12 → misclassified
  w3 = w2 + x_pos = (-2, -4) + (10, -2) = (8, -6)

- x_neg: w3 · x_neg = 8*12 + (-6)*2 = 96 - 12 = 84, y = -1
  y (w3 · x_neg) = -84 ≤ 0 → misclassified
  w4 = w3 + (-1) x_neg = (8, -6) + (-12, -2) = (-4, -8)

3rd pass:
- x_pos misclassified → w5 = w4 + x_pos = (-4, -8) + (10, -2) = (6, -10)
- x_neg misclassified → w6 = w5 + (-1) x_neg = (6, -10) + (-12, -2) = (-6, -12)

4th pass:
- x_pos misclassified → w7 = w6 + x_pos = (-6, -12) + (10, -2) = (4, -14)
- x_neg misclassified → w8 = w7 + (-1) x_neg = (4, -14) + (-12, -2) = (-8, -16)

5th pass:
- x_pos: w8 · x_pos = (-8)*10 + (-16)*(-2) = -80 + 32 = -48 → misclassified
  w9 = w8 + x_pos = (-8, -16) + (10, -2) = (2, -18)

Check:
- x_neg: w9 · x_neg = 2*12 + (-18)*2 = 24 - 36 = -12, y = -1
  y (w9 · x_neg) = -1 * (-12) = 12 > 0 → correctly classified.
- x_pos: w9 · x_pos = 2*10 + (-18)*(-2) = 20 + 36 = 56 > 0 → correctly classified.

At w9 = (2, -18), both points are correctly classified, so the algorithm stops.

Number of updates: 9

Sequence of weight vectors:
w0 = (0, 0)
w1 = (10, -2)
w2 = (-2, -4)
w3 = (8, -6)
w4 = (-4, -8)
w5 = (6, -10)
w6 = (-6, -12)
w7 = (4, -14)
w8 = (-8, -16)
w9 = (2, -18)
"""

# Optional: small check with code
import numpy as np

x_pos = np.array([10.0, -2.0])
x_neg = np.array([12.0,  2.0])
points = [x_pos, x_neg]
labels = [1, -1]

w = np.array([0.0, 0.0])
history = [w.copy()]

for _ in range(20):  # safety limit
    changed = False
    for x, y in zip(points, labels):
        if y * np.dot(w, x) <= 0:
            w = w + y * x
            history.append(w.copy())
            changed = True
    if not changed:
        break

print("Perceptron weight sequence:")
for i, w_i in enumerate(history):
    print(f"w{i} =", tuple(w_i))


### Problem 4: Reconstructing the Weight Vector

Given the log of Perceptron updates:

| x | y | count |
|---|---|--------|
| (0, 0, 0, 0, 4) | +1 | 2 |
| (0, 0, 6, 5, 0) | +1 | 1 |
| (3, 0, 0, 0, 0) | -1 | 1 |
| (0, 9, 3, 6, 0) | -1 | 1 |
| (0, 1, 0, 2, 5) | -1 | 1 |

Assume learning rate = 1 and initial weight $w_0 = (0, 0, 0, 0, 0)$.

Compute the final weight vector after all updates.

In [ ]:
"""Problem 4 – Reconstructing the Weight Vector

We start with initial weight
w0 = (0, 0, 0, 0, 0)
and learning rate 1.

Each row in the table records an example x, its label y, and how many times this example caused an update. Each update is:
    w ← w + y x
So if an example is used 'count' times, its total contribution is:
    Δw = count · y · x

Row 1: x = (0, 0, 0, 0, 4), y = +1, count = 2
Δw1 = 2 · (+1) · (0, 0, 0, 0, 4) = (0, 0, 0, 0, 8)
w1 = w0 + Δw1 = (0, 0, 0, 0, 8)

Row 2: x = (0, 0, 6, 5, 0), y = +1, count = 1
Δw2 = 1 · (+1) · (0, 0, 6, 5, 0) = (0, 0, 6, 5, 0)
w2 = w1 + Δw2 = (0, 0, 6, 5, 8)

Row 3: x = (3, 0, 0, 0, 0), y = −1, count = 1
Δw3 = 1 · (−1) · (3, 0, 0, 0, 0) = (−3, 0, 0, 0, 0)
w3 = w2 + Δw3 = (−3, 0, 6, 5, 8)

Row 4: x = (0, 9, 3, 6, 0), y = −1, count = 1
Δw4 = 1 · (−1) · (0, 9, 3, 6, 0) = (0, −9, −3, −6, 0)
w4 = w3 + Δw4 = (−3, −9, 3, −1, 8)

Row 5: x = (0, 1, 0, 2, 5), y = −1, count = 1
Δw5 = 1 · (−1) · (0, 1, 0, 2, 5) = (0, −1, 0, −2, −5)
w5 = w4 + Δw5 = (−3, −10, 3, −3, 3)

Therefore, the final Perceptron weight vector is:
w_final = (−3, −10, 3, −3, 3)
"""

# Optional: verify with code
import numpy as np

w = np.zeros(5, dtype=float)

rows = [
    ((0, 0, 0, 0, 4),  1, 2),
    ((0, 0, 6, 5, 0),  1, 1),
    ((3, 0, 0, 0, 0), -1, 1),
    ((0, 9, 3, 6, 0), -1, 1),
    ((0, 1, 0, 2, 5), -1, 1),
]

for x, y, count in rows:
    x = np.array(x, dtype=float)
    w += count * y * x

print("Computed final weight vector:", w)


### Problem 5: Visualizing Perceptron Convergence

Implement a Perceptron on a small 2D dataset with positive and negative examples.

- Plot the data points.
- After each update, visualize the decision boundary.
- Show how it converges to a stable separator.

In [ ]:
"""Problem 5 – Visualizing Perceptron Convergence

In this problem, I construct a simple 2D, linearly separable dataset, train a Perceptron with a bias term, and plot the decision boundary after each update. This shows how the separating hyperplane moves as the algorithm converges.
"""

import numpy as np
import matplotlib.pyplot as plt

# Simple linearly separable 2D dataset
X_pos = np.array([[2, 3],
                  [3, 4],
                  [4, 3]])
X_neg = np.array([[2, 1],
                  [3, 0],
                  [4, 1]])

X = np.vstack([X_pos, X_neg])
y = np.hstack([np.ones(len(X_pos)), -np.ones(len(X_neg))])

# Add bias term: x' = [x1, x2, 1]
X_aug = np.hstack([X, np.ones((X.shape[0], 1))])

# Initialize weights (including bias)
w = np.zeros(3)
history = [w.copy()]

max_epochs = 20
for epoch in range(max_epochs):
    changed = False
    for i in range(len(X_aug)):
        if y[i] * (X_aug[i] @ w) <= 0:
            # Perceptron update
            w = w + y[i] * X_aug[i]
            history.append(w.copy())
            changed = True
    if not changed:
        break

print("Total number of updates:", len(history) - 1)
print("Final weight vector:", history[-1])

# Visualization of decision boundary after each update
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1

for t, w_t in enumerate(history):
    plt.figure()
    plt.scatter(X_pos[:, 0], X_pos[:, 1], marker='o', label='positive')
    plt.scatter(X_neg[:, 0], X_neg[:, 1], marker='x', label='negative')

    # decision boundary: w1 * x1 + w2 * x2 + w3 = 0 ⇒ x2 = -(w1 * x1 + w3) / w2
    if abs(w_t[1]) > 1e-6:
        xs = np.linspace(x_min, x_max, 100)
        ys = -(w_t[0] * xs + w_t[2]) / w_t[1]
        plt.plot(xs, ys, label=f'after update {t}')
    else:
        # Vertical line if w2 is zero (rare in this toy example)
        xs = np.full(100, -w_t[2] / (w_t[0] + 1e-8))
        ys = np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 100)
        plt.plot(xs, ys, label=f'after update {t}')

    plt.title(f"Perceptron decision boundary – update {t}")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.legend()
    plt.show()
